# Tutorial 2: Intro to pandas for data science

This tutorial goes over how to use the pandas library for working with large sets of data. It is is adapted from Keith Galli's tutorial on pandas: https://github.com/KeithGalli/pandas/tree/master 

---

## 2.1 Importing data

The pandas library allows us to work with two important types of data:
1) **Series**: a one-dimensional labeled array holding data of any type. It's similar to a list, but each element has an associated label (or 'index') associated with it.
2) **DataFrame**: a two-dimensional data structure array. You can think of it like a spreadsheet or table, with rows and columns. 

\
Start by importing the library:

In [ ]:
import pandas as pd

\
We can create a dataframe from any delimited, text-based file. Let's load the **pokemon_data.csv** file as a dataframe:

In [ ]:
#the 'sep' argument specifies the delimiter type. For tab-separated files, use '\t'
df = pd.read_table('pokemon_data.csv', sep=',') 

#to view our dataframe:
df

To view only the first or last N rows, use `df.head(N)` or `df.tail(N)`. 

---

## 2.2 Viewing rows and columns

Some basic commands for accessing specific data by row/column:

In [ ]:
#displaying data for one column (note that this returns a pandas series):
df['Name']

In [ ]:
#multiple columns (note the use of double square brackets):
df[['#', 'Name']]

In [ ]:
#displaying data for one row:
df.loc[2]

In [ ]:
#multiple rows:
df.loc[[4,5,6]]

In pandas, rows are referred to by **index**. By default it is numerical. More on this later.

---

## 2.3 Conditional filtering

We can combine column/row indexing with **boolean** operators to filter specific rows:

In [ ]:
df.loc[df['Name']=='Magikarp'] 

The entire expression evaluates to "the rows in `df` where the value in the `Name` column is equal to `Magikarp`". 

Note that the `==` operator checks if two values are equal, and returns `True` if they do, `False` otherwise.\
Guide to comparison operators: https://www.w3schools.com/python/gloss_python_comparison_operators.asp 

We can also chain together multiple boolean operators using **logical** operators:

In [ ]:
#filtering for multiple conditions:
df.loc[(df['Attack']>100) & (df['Defense']<50)] 

In the above example, `&` is short for the `AND` operator: the statement inside the square brackets will only return `True` if both statements in the round brackets return `True`. Note that you have to group the individual statements in round brackets when using logical operators. 

Try replacing `&` with `|` (short for the `OR` operator) to see what happens. 

Finally, putting `~` in front of a boolean statement (the `NOT` operator) returns the opposite of the original result. So `~(df['Attack']>100)` will return `True` for all values 100 and below, and `False` for all values above 100. Note the extra round brackets. The `!=` operator can be used as a shorthand for 'not equal to' instead.

---

### Exercise #1:

Filter and display the data from `df` of only Grass pokemon (Type 1 or Type 2) with HP > 50.\
Hint: you may need an extra set of round brackets to ensure correct order of operations.

In [ ]:
### YOUR CODE HERE ###

---

## 2.4 Modifying data

In the previous sections, even though the output cells showed modified versions of `df`, we didn't actually make changes to the dataframe itself.

To do so, we have to re-write the original dataframe with `df =` followed by the function/command. **Be very careful doing this**, because overwriting the original data may cause unintended consequences when trying to re-run your code. Often, it is better to use the `copy()` command when executing a new cell. 

In [ ]:
new_df = df.copy()
new_df = new_df.loc[(new_df['Attack']>100) & (new_df['Defense']<50)] 

#we have changed new_df, so viewing it shows the filtered data:
new_df

\
Notice that the `df.copy()` syntax is similar to the string/list methods covered in Tutorial 1. Series and dataframe functions work the same way, where you specify the object before the function, and then the parameters of the function within the brackets. Here are some other useful examples:

In [ ]:
df2 = df.copy()

#dropping columns and rows:
df2 = df2.drop('Legendary', axis=1) #the axis argument specifies column (1) or row (0)
df2 = df2.drop([2, 3, 4], axis=0) #multiple columns or rows can be dropped by passing a list

#sorting by column values:
df2 = df2.sort_values(by='Type 1', ascending=False) #specifying ascending=False sorts in reverse order
df2 = df2.sort_values(by=['Type 1', 'Type 2'], ascending=[False, True]) #multi-column sorting

df2

However, operations that define a new column, or change an existing column, **will** modify the original dataframe. Some examples:

In [ ]:
df2 = df.copy()

#creating a copy of an existing column
df2['Name_copy'] = df2['Name']

#adding 10 to every number in a numerical column (specifying an existing column --> it will overwrite the current data)
df2['HP'] = df2['HP'] + 10

#concatenating two text columns:
df2['All_types'] = df2['Type 1'] + ' ' + df2['Type 2']

#an example of using string methods
#note that you must specify "str" as the data type, otherwise it thinks you are trying to split the column
df2['first_name'] = df2['Name'].str.split(' ', expand=True)[0]

df2

---

### Exercise #2: 

Read the documentation for `df.sum()`: \
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html 


Using `df.sum()`, create a new column in `df` (or a copy of `df`) that shows the sum of all stats (all numerical columns) for all pokemon.

Hint: pandas functions often have many optional parameters. If not specified, the default value is used (listed in the documentation). This exercise only requires one specified parameter within `df.sum()`. 


In [ ]:
### YOUR CODE HERE ###

---

## 2.5 Aggregating statistics

The `df.groupby()` method splits the data based on a category, applies a function, and combines the results.

The following is an example of calculating the number of pokemon of each Generation using `.groupby()` and `.count()`:

In [ ]:
df.groupby('Generation').count()

Note that the resulting dataframe is automatically re-indexed by Generation.

We can also specify a specific column to apply the function to. Can you guess what the following code does?

In [ ]:
df.groupby('Generation')['HP'].sum()

Note that passing a single column returns a series and not a dataframe.

\
We can also create hierarchical groupings by passing a list of columns into `.groupby()`:

In [ ]:
df.groupby(['Legendary','Generation'])[['Attack','Defense']].mean()

---

### Exercise #3:

Create a new dataframe that shows the total number of pokemon in each **Generation**, by each type (**Type 1**). There should only be **one** column for counts. 


Hints: 
1) The column specified for the `.count()` function is somewhat arbitrary, as long as it doesn't contain any NaN values. 
2) Remember that passing a single column to a function returns a series. To convert it to a dataframe, use `series.to_frame()`.

In [ ]:
## YOUR CODE HERE ##

---

## 2.6 Combining data 

The following code does the same grouping as in Exercise 3, but with two dataframes: one for **Type 1** counts and one for **Type 2** counts. 

There is also some additional formatting with `df.rename()` but you dont need to know the details for now.

In [ ]:
df_1 = df.copy().rename({'#':'Type1_counts', 'Type 1':'Type'},axis=1)
df_1 = df_1.groupby(['Generation','Type'])['Type1_counts'].count().to_frame()

df_2 = df.copy().rename({'#':'Type2_counts', 'Type 2':'Type'},axis=1)
df_2 = df_2.groupby(['Generation','Type'])['Type2_counts'].count().to_frame()

display(df_1) #display() is like print(), if you want to show multiple dataframes
display(df_2)

Lets say I wanted to add the Type 1 and Type 2 counts from `df_1` and `df_2`. If they had the exact same sorted index, I could just use `df_1['T1_counts'] + df_2['T2_counts']` to return a series with the totals. However, not all Types are present in both dataframes, which makes matching up the rows tricky.

The `df.join()` function solves this problem, as it matches up values by the labels of a given index or column:

In [ ]:
df_3 = df_1.join(df_2, on=['Generation', 'Type'], how='outer')
df_3

The `on=` parameter tells it the common index(es) or column(s) to join on. They have to be the same name in both dataframes, which is why I used `.rename()` earlier.


The `how=` parameter is set to `outer` here, so it includes **all** index values (i.e. values that are present in `df_1` OR `df_2`). Specifying `inner` would only keep index values that are shared between both dataframes, while `left` and `right` would keep only values from `df_1` and `df_2` respectively. 

\
From here, obtaining the total counts is simple:

In [ ]:
df_4 = df_3.copy()
df_4 = df_4.fillna(0) #replaces NaN values with 0 to enable addition
df_4['Counts'] = df_4['Type1_counts'] + df_4['Type2_counts']
df_4 = df_4[['Counts']] #filtering for only the total counts column

df_4

---

## 2.7 Reshaping

The most common reshaping function is `df.pivot()`. To use it on `df_4` we first need to use `df.reset_index()` which takes the current index columns `[Generation, Type]` and turns them into regular data columns. 

In [ ]:
df_5 = df_4.reset_index().pivot(index='Generation', columns='Type', values='Counts')
df_5

The `.pivot()` function creates a new dataframe where the row and column labels are specified by the `index` and `columns` parameters respectively. The `values` parameter is the data to fill the new dataframe with. 

All three parameters must be columns from the original dataframe, which is why we used `.reset_index()`, which turns the current index into a column. 

---

## 2.8 Applying functions

Recall that there are many built-in functions for series and dataframes. However, we can also define our own functions and use them with `.apply()`: 

In [ ]:
#this function divides a value by 10 if it is greater than 10
def div_10(x):
    if x > 10:
        return x/10
    else:
        return x

df_5['Bug'].apply(div_10)

\
Note that `series.apply()` takes a function that works on a **single value** and applies it to **each value in the series**. 

For `dataframe.apply()`, you must pass a function that works on a **series** (or list) to apply it to **each column of the dataframe**.

In [ ]:
#this function divides each element in a series by the maximum value in the series
def scale_max(col):
    return col/col.max()

df_5.apply(scale_max)

If you wanted to use `.apply()` on rows instead of columns, you would specify `axis=1` in the parameters.

---

### Exercise #4



The previous examples didn't modify `df_5`, so it should still hold the original pokemon counts. 

Using `df.apply()`, create a new dataframe from `df_5` where all values are scaled as a **percentage** of the total values for each Generation.\
Hint: you can check your work by seeing if row values add up to 100. 


In [ ]:
### YOUR CODE HERE ###

---

## 2.9 Exporting data

This concludes Tutorial 2. We will be using the dataframe from Exercise 4 in the next tutorial, so export it using the following:

In [ ]:
YOUR_DATAFRAME.to_csv('YOUR_NAME_pokemon_data.csv')

Please check that the output is correct (see Tutorial answers notebook) before exporting.

---

## 2.10 Extras

Many pandas functions are not covered in this tutorial, but are commonly used in general data science tasks. Knowing these in detail is **not required**, although a couple may come up in the assignment. If they do, I will explicitly state which question(s) they apply to and link the appropriate examples & documentation. Most of the time, they are fairly intuitive and easy to figure out. 

### Indexing and data types

`df.len()` returns the number of rows for a dataframe.\
`df.columns` and `df.index` returns column and row names. \
`series.describe()` returns basic statistical measures of a series.\
`series.unique()` returns the unique values of a series.\
`df.dtypes` returns a series of the *type* of data in each column (int, string, etc)


You can turn any series into a list with the `.to_list()` function.\
You can also convert series to different data types. For example, `.to_string()` converts all data in a series to string type, regardless of original type. Often needed to ensure the data type is compatible with a function. 


### Filtering

`df.query()` is an alternate way of boolean indexing by column value.\
`df.select_dtypes()` can be used to filter for only columns of a specific data type.

`series.drop_duplicates()` returns a series with duplicate rows removed, keeping the first occurence. `keep='last'` will keep the last occurence instead.\
`series.isin()` returns a series with only elements contained in a list.\
`series.where()` is a faster way of performing operations on a series based on a condition.



Many string methods can be used on series to filter for conditions. These include `str.startswith()`, `str.endswith()`, and `str.contains()`.


### Combining data

`pd.merge()` is similar to `.join()`.\
`pd.concat()` enables joining rows instead of columns.


### Reshaping 

This guide is useful for visualizing other types of data reshaping: https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping \
`df.stack()`, `df.unstack()`, `df.melt()`, and `df.explode()` are common.


### Other libraries
Most of the time, pandas is used alongside **NumPy**
(https://numpy.org/doc/stable/user/quickstart.html#basic-operations). The NumPy library enables data manipulation of arrays, which is another data type. It also has some cool functions such as `np.random()` to generate random numbers and `np.arange()` to generate numerical series. 